# Real World Application 
- Customer Service bots (simple queries -> Qwen3 , complex issues -> GPT-OSS)
- Research assistants (quick facts -> Qwen3 , analysis -> GPT-OSS)

In [75]:
from langchain.agents import create_agent , AgentState
from langchain_ollama import ChatOllama
from langgraph.runtime import Runtime
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_experimental.tools import PythonREPLTool
from langchain.tools import tool
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

In [76]:
math = PythonREPLTool()
web_search = DuckDuckGoSearchRun()

@tool
def analyze_data(data: list, operation: str = "df.describe()") -> str:
    """
    Dynamic pandas data analysis tool.
    """
    try:
        import pandas as pd
        import numpy as np
    except ImportError as e:
        return f"Missing dependency: {e}"

    if not data or not isinstance(data, list):
        return "Data must be a non-empty list."

    try:
        df = pd.DataFrame(data)
    except Exception as e:
        return f"Failed to create DataFrame: {str(e)}"

    if df.empty:
        return "Empty DataFrame created."

    namespace = {"df": df, "pd": pd, "np": np}
    try:
        result = eval(operation, {"__builtins__": {}}, namespace)
    except SyntaxError:
        return "Invalid operation syntax."
    except Exception as e:
        return f"Operation error: {str(e)}"

    if isinstance(result, (pd.DataFrame, pd.Series)):
        return result.to_string()
    else:
        return str(result)

## Model Selection Function

In [96]:
llm1 = ChatOllama(
    model="qwen3:4b",
    temperature=0.1
)
llm2 = ChatOllama(
    model="qwen3:4b",
    temperature=0.0
)

In [97]:
tools_list = [analyze_data, web_search, math]

@wrap_model_call
def select_model(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    
    message_count = len(request.state["messages"])

    if message_count < 10:
        print(f"[Agent] Using qwen3:4b for {message_count} messages (efficient)")
        llm = llm1
    else:
        print(f"[Agent] Using qwen3:4b for {message_count} messages (advanced)")
        llm = llm2

    request.model = llm
    return handler(request)

print("✅ Model selection function ready!")

✅ Model selection function ready!


## Create Dynamic Agent 

In [98]:
agent = create_agent(
    model=llm1,  # Default model
    tools=tools_list,
    middleware=[select_model]
)

### Test 1 - Short Conversaion

In [99]:
res_1 = agent.invoke(
    {"messages": [{"role": "user", "content": "Search for me about latest Ai News"}]}
)

[Agent] Using qwen3:4b for 1 messages (efficient)
[Agent] Using qwen3:4b for 3 messages (efficient)


In [100]:
res_1["messages"][-1].pretty_print()

================================== Ai Message ==================================

Here are the latest AI news highlights based on recent reports:

- **Reuters** covers AI breakthroughs, technology trends, regulatory developments, ethical concerns, business impacts, and global implications (updated 3 days ago).
- **AI News** provides insights into industry trends, research advancements, product launches, and emerging technologies (curated from top sources).
- Recent updates highlight active coverage of AI research breakthroughs and industry developments, though some search results mention future dates (e.g., June 25, 2025), which may be placeholders or system-generated timestamps.

This summary reflects the most current AI-related news trends as reported by major outlets and specialized platforms. Let me know if you'd like deeper analysis on a specific topic!


## Advanced Model Selection

In [105]:
@wrap_model_call
def intelligent_model_selection (request: ModelRequest, handler) -> ModelResponse : 
    """intelligent model selection based on multiple factors."""
    messages = request.state["messages"]
    message_count = len(messages)

    total_length = sum(
        len(str(msg.content))
        for msg in messages
        if hasattr(msg,"content") and msg.content
    )


    complex_keywords = ["analysis" , "research" , "detailed" , "comperehensive" , "complex" ,"strategy"]

    has_complex_content = any(
        keyword in str(msg.content).lower()
        for msg in messages
        for keyword in complex_keywords
        if hasattr(msg,"content") and msg.content
    )

    if total_length > 3000 or has_complex_content or message_count > 8 : 
        print(message_count , total_length , has_complex_content)
        request.model = ChatOllama(model="qwen3:4b" , temperature=0.0 , num_predict=2500)
        return handler(request)
    else : 
        print(message_count , total_length , has_complex_content)
        request.model = ChatOllama(model="qwen3:4b" , temperature=0.1 , num_predict=1000)
        return handler(request)

In [106]:
agent2 = create_agent(
    model=llm1,
    tools=[web_search],
    middleware=[intelligent_model_selection]
)

In [110]:
res_2 = agent2.invoke(
    {"messages": [{"role": "user", "content": "Search for Machine learning"}]}
)

1 27 False
3 1289 False


In [111]:
res_2["messages"][-1].pretty_print()

================================== Ai Message ==================================

Machine learning (ML) is a branch of artificial intelligence focused on developing statistical algorithms that enable computers to learn from data without explicit programming. It allows systems to improve at tasks through experience, such as pattern recognition and decision-making. Key applications span healthcare, finance, and gaming, driving innovation across industries. Beginners can explore foundational skills through free resources like MIT Open Learning and Coursera courses, which provide practical insights into building and applying machine learning models.
